<a href="https://colab.research.google.com/github/gregmeldrum/localLLM/blob/main/free-tier-replacement-for-copilot/Deepseek_Coder6_7B_Server_as_copilot_replacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Deepseek Coder 6.7B Server as Copilot Replacement

This colab starts an instance of the [Deepseek Coder](https://huggingface.co/deepseek-ai/deepseek-coder-6.7b-instruct) model (6 bit quantized) and makes the model publicly available with ngrok. Look for the ngrok public url at the end of the output for step 2. You can sign up to [ngrok](https://ngrok.com/) for free if you don't already have an account.

To verify that the server is running, check the OpenAPI spec at https://dd26-34-142-187-115.ngrok-free.app/docs (replace the ngrok URL with your own and add `/docs` at the end)

In order to use this with github copilot on VSCode, open the settings for the github copilot plugin, and add the following to the advanced settings:
```
{
    "github.copilot.advanced": {
        "debug.testOverrideProxyUrl": "https://dd26-34-142-187-115.ngrok-free.app",
        "debug.overrideProxyUrl": "https://dd26-34-142-187-115.ngrok-free.app"
    }
}
```

Replace the ngrok url above with your own.

This colab uses a 6bit quantized gguf version of the 6.7B instruct model.

This colab uses [llama-cpp-server](https://github.com/abetlen/llama-cpp-python) to run the model and serve the copilot interface and [ngrok](https://ngrok.com/) to give it a public URL.

In [ ]:
#@title <b>Step 1 - Enter your ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken</b> and <b>run this cell</b>
authtoken = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX' # @param {type:"string"}

## **Step 2: Run the model**
Look for the ngrok URL at the end of the output

In [ ]:
# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

import os
os.environ["AUTHTOKEN"] = authtoken

# Download the model from hugging face (6 bit quantized version - by default ollama uses 4 bit)
!wget https://huggingface.co/TheBloke/deepseek-coder-6.7B-instruct-GGUF/resolve/main/deepseek-coder-6.7b-instruct.Q6_K.gguf?download=true -O /content/deepseek-coder-6.7b-instruct.Q6_K.gguf
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python[server] --force-reinstall --upgrade --no-cache-dir
!nohup bash -c "python -m llama_cpp.server --n_ctx 4096 --n_gpu_layers 50 --model /content/deepseek-coder-6.7b-instruct.Q6_K.gguf --host 0.0.0.0" &

# Start ngrok
!pip install pyngrok==7.0.1
!ngrok config add-authtoken $AUTHTOKEN
!ngrok http --log stderr 8000